In [1]:
import pandas as pd
import altair as alt

In [2]:
profile= pd.read_csv(
    #'Results/Batch4/CaseB/dispatch_10_1_2_price_improved.csv',
    #'Results/Batch4/CaseB/dispatch_10_1_2_price_limit.csv',
    'Results/Batch5/CaseB/co2_3_1_2.csv',
    #'Results/Batch5/CaseB/price_3_1_2.csv',

    sep= ',',
    index_col=False,
)
#profile['SoC']= ((profile['SoC']/20000)*1000).astype('float')
profile['SoC']= ((profile['SoC']/6000)*100).astype('float')

# p_solar=    10
# p_bess=     1
# t_bess=     2
# e_bess=     p_bess * t_bess
p_solar=    10
p_bess=     3
t_bess=     2
e_bess=     p_bess * t_bess

efficiency_charge = 0.85
efficiency_discharge = 0.90
efficiency_inverter = 0.97


week= 24

#profile.info()
#profile[47:96]

In [3]:
profile['P_PV_to_Grid']=    profile['P_PV_to_Grid']     * efficiency_inverter
profile['P_PV_to_Load']=    profile['P_PV_to_Load']     * efficiency_inverter
profile['P_PV_to_BESS']=    profile['P_PV_to_BESS']     * efficiency_charge     * efficiency_inverter
profile['P_BESS_to_Grid']=  profile['P_BESS_to_Grid']   * efficiency_discharge  * efficiency_inverter
profile['P_BESS_to_Load']=  profile['P_BESS_to_Load']   * efficiency_discharge  * efficiency_inverter
profile['P_Grid_to_BESS']=  profile['P_Grid_to_BESS']   * efficiency_charge     * efficiency_inverter

profile= profile[:8760]
#profile

In [4]:

price = pd.read_csv('Data/PriceCurve_SE4_2021.csv', sep = ';')
co2_pro = pd.read_csv('Data/production_emissions.csv')
co2_con = pd.read_csv('Data/consumption_emissions.csv')
pv = pd.read_csv('Data/oskarshamnpvprod.csv')
#load = pd.read_csv('Data/LoadCurveo.csv', sep = ',')
load = pd.read_csv('Data/LoadCurve_new.csv', sep=';')
load['Load'] = load['Load'].apply(lambda x: x.replace(',', '.')).astype(float)

data = load
data['Price'] = price['Grid_Price']
data['CO_2_eq'] = co2_pro['carbon_intensity_production_avg']
data['solar_PV'] = pv

#Converting from MWh to kWh
data['Load']= (data['Load'] * 1000)
#solar data is already in kW, but needs to be multiplied by the scaling factor
data['solar_PV']= (data['solar_PV']) * p_solar
#data['Price']= (data['Price'] / 1000)
# CO_2_eq is also already in kWh
#data['CO_2_eq']= (data['CO_2_eq'] / 1000)

data['Hour']= (data['Hour']).astype('int')

#data.head(48)
data= data.drop(data.index[0])
data= ((data.reset_index()).drop(columns= {'Hour'})).rename(columns= {'index': 'Hour'})
data.head(10)

,Hour,Load,Price,CO_2_eq,solar_PV
0,1,1008.0,24.35,37.48,0.000000
1,2,1086.0,23.98,37.29,0.000000
2,3,1128.0,23.72,37.22,0.000000
3,4,1032.0,23.73,37.50,0.000000
4,5,1014.0,24.06,37.76,0.000000
5,6,1008.0,24.44,38.65,0.000000
6,7,1044.0,24.93,38.93,0.000000
7,8,1104.0,24.90,38.83,0.000000
8,9,1056.0,25.07,38.72,0.000000
9,10,1050.0,25.63,38.75,0.156337


In [5]:
# Select only the columns of interest
columns_of_interest = ['Hour','P_PV_to_Load','P_PV_to_Grid','P_PV_to_BESS','P_PV_curtailment','P_BESS_to_Load','P_Grid_to_Load']
subset_data_2= profile[columns_of_interest]
subset_data_2= subset_data_2.fillna(0)  

# Melt the DataFrame to long format
melted_data = subset_data_2.melt(id_vars=['Hour'], var_name='source', value_name='value')

alt.Chart(melted_data[4030:4201]).mark_area().encode(
    x= alt.X('Hour'),
    y= alt.Y(
        'value:Q', stack= 'zero'
    ),
    color= alt.Color(
        'source:N'
    )
)

alt.Chart(...)

In [6]:
subset_data_2

,Hour,P_PV_to_Load,P_PV_to_Grid,P_PV_to_BESS,P_PV_curtailment,P_BESS_to_Load,P_Grid_to_Load
0,0,0.0,0.0,0.0,0.0,0.0,1020.0
1,1,0.0,0.0,0.0,0.0,0.0,1008.0
2,2,0.0,0.0,0.0,0.0,0.0,1086.0
3,3,0.0,0.0,0.0,0.0,0.0,1128.0
4,4,0.0,0.0,0.0,0.0,0.0,1032.0
...,...,...,...,...,...,...,...
8731,8731,0.0,0.0,0.0,0.0,0.0,2510.7
8732,8732,0.0,0.0,0.0,0.0,0.0,2496.7
8733,8733,0.0,0.0,0.0,0.0,0.0,2575.0
8734,8734,0.0,0.0,0.0,0.0,0.0,2427.5


In [7]:
subset_data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8736 entries, 0 to 8735
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Hour              8736 non-null   int64  
 1   P_PV_to_Load      8736 non-null   float64
 2   P_PV_to_Grid      8736 non-null   float64
 3   P_PV_to_BESS      8736 non-null   float64
 4   P_PV_curtailment  8736 non-null   float64
 5   P_BESS_to_Load    8736 non-null   float64
 6   P_Grid_to_Load    8736 non-null   float64
dtypes: float64(6), int64(1)
memory usage: 477.9 KB


In [8]:
melted_data[4030:4201]['source'].unique()

array(['P_PV_to_Load'], dtype=object)

In [9]:
data[:8760].describe()

,Hour,Load,Price,CO_2_eq,solar_PV
count,8759.000000,8759.000000,8759.000000,8759.000000,8759.000000
mean,4380.000000,2316.237653,66.028333,37.908945,6.668030
std,2528.649837,461.628870,59.372347,4.144494,12.382389
min,1.000000,524.400000,-1.970000,27.090000,0.000000
25%,2190.500000,2244.000000,33.970000,34.760000,0.000000
50%,4380.000000,2442.000000,50.930000,37.070000,0.000000
75%,6569.500000,2585.950000,79.190000,40.780000,6.925150
max,8759.000000,3080.200000,626.060000,50.660000,53.854800


In [10]:
print(f'Price Variance: {data[:8760].Price.var()}')
print(f'Price Mean Value: {data[:8760].Price.mean()}')
print(f'CO2eq Variance: {data[:8760].CO_2_eq.var()}')
print(f'CO2eq Mean Value: {data[:8760].CO_2_eq.mean()}')
print('-'*50)
print(f'Price CoV: {data[:8760].Price.var()/data[:8760].Price.mean()}')
print(f'CO2eq CoV: {data[:8760].CO_2_eq.var()/data[:8760].CO_2_eq.mean()}')
print('-'*50)
print(f'Price Standard Deviation: {data[:8760].Price.std()}')
print(f'CO2eq Standard Deviation: {data[:8760].CO_2_eq.std()}')
print('-'*50)
print(f'Covariance: {data[:8760].Price.corr(data[:8760].CO_2_eq)}')

Price Variance: 3525.0756259949626
Price Mean Value: 66.02833314305285
CO2eq Variance: 17.17683414621899
CO2eq Mean Value: 37.90894508505537
--------------------------------------------------
Price CoV: 53.38731811323716
CO2eq CoV: 0.4531076796697916
--------------------------------------------------
Price Standard Deviation: 59.37234731754306
CO2eq Standard Deviation: 4.144494437952473
--------------------------------------------------
Covariance: 0.3447220392459419
